In [1]:
import sys
sys.path.append("./work/streamify")

from operators.streaming import SparkStreaming
import pyspark.sql.functions as F

In [2]:
spark = SparkStreaming.get_instance(app_name="Streamify Spark Streaming")

In [21]:
spark

# Read data from MinIO

In [11]:
rental_df = (spark.read
                    .format("delta")
                    .load("s3a://lakehouse/streaming/streamify/rental")
                    .dropDuplicates())
print(rental_df.count())
rental_df.printSchema()

66
root
 |-- rental_id: integer (nullable = true)
 |-- rental_date: string (nullable = true)
 |-- inventory_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- return_date: string (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- last_update: string (nullable = true)
 |-- rental_year: integer (nullable = true)
 |-- rental_month: integer (nullable = true)



In [12]:
rental_df.show(5)

+---------+-------------------+------------+-----------+-------------------+--------+-------------------+-----------+------------+
|rental_id|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|rental_year|rental_month|
+---------+-------------------+------------+-----------+-------------------+--------+-------------------+-----------+------------+
|   281500|2010-01-03 09:27:25|        2586|        446|2010-01-05 07:42:24|      64|2010-01-02 08:33:44|       2010|           1|
|   281483|2010-01-02 07:26:19|        3135|        591|2010-01-07 05:53:43|      75|2010-01-01 10:18:55|       2010|           1|
|   281536|2010-01-09 00:38:36|           3|        255|2010-01-13 17:26:34|     115|2010-01-04 10:29:16|       2010|           1|
|   281494|2010-01-03 01:29:12|        1229|        312|2010-01-04 00:13:16|      71|2010-01-02 03:04:21|       2010|           1|
|   281489|2010-01-02 15:15:09|        2387|        472|2010-01-06 13:17:17|      1

In [3]:
payment_df = (spark.read
                    .format("delta")
                    .load("s3a://lakehouse/streaming/streamify/payment")
                    .drop("last_update")
                    .dropDuplicates())
print(payment_df.count())
payment_df.printSchema()

64
root
 |-- rental_id: integer (nullable = true)
 |-- payment_date: string (nullable = true)
 |-- payment_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- staff_id: integer (nullable = true)
 |-- payment_year: integer (nullable = true)
 |-- payment_month: integer (nullable = true)



In [14]:
join_df = payment_df.join(rental_df, on=["rental_id"], how="inner")
print(join_df.count())
join_df.show(5)

64
+---------+-------------------+----------+-----------+------+--------+------------+-------------+-------------------+------------+-----------+-------------------+--------+-------------------+-----------+------------+
|rental_id|       payment_date|payment_id|customer_id|amount|staff_id|payment_year|payment_month|        rental_date|inventory_id|customer_id|        return_date|staff_id|        last_update|rental_year|rental_month|
+---------+-------------------+----------+-----------+------+--------+------------+-------------+-------------------+------------+-----------+-------------------+--------+-------------------+-----------+------------+
|   281500|2010-01-05 07:42:24|     40261|        446|  39.7|      64|        2010|            1|2010-01-03 09:27:25|        2586|        446|2010-01-05 07:42:24|      64|2010-01-02 08:33:44|       2010|           1|
|   281483|2010-01-07 05:53:43|     40244|        591|  39.7|      75|        2010|            1|2010-01-02 07:26:19|        3135

In [6]:
customer_df = spark.read.csv("s3a://lakehouse/csv/customer.csv", header=True, inferSchema=True, samplingRatio=0.1)
address_df = spark.read.csv("s3a://lakehouse/csv/address.csv", header=True, inferSchema=True, samplingRatio=0.1)
city_df = spark.read.csv("s3a://lakehouse/csv/city.csv", header=True, inferSchema=True, samplingRatio=0.1)
country_df = spark.read.csv("s3a://lakehouse/csv/country.csv", header=True, inferSchema=True, samplingRatio=0.1)

In [10]:
spark.sql("CREATE DATABASE streamify").show()

++
||
++
++



In [11]:
customer_df.write.format("delta").mode("overwrite").saveAsTable("streamify.customer")
address_df.write.format("delta").mode("overwrite").saveAsTable("streamify.address")
city_df.write.format("delta").mode("overwrite").saveAsTable("streamify.city")
country_df.write.format("delta").mode("overwrite").saveAsTable("streamify.country")

# Synchronize data between MinIO and Trino

In [28]:
spark.sql("USE streamify")
# spark.sql("DROP TABLE IF EXISTS delta_payment")
# spark.sql("DELETE FROM hive_payment")
spark.sql("SHOW TABLES").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|streamify|     address|      false|
|streamify|        city|      false|
|streamify|     country|      false|
|streamify|    customer|      false|
|streamify|hive_payment|      false|
+---------+------------+-----------+



In [32]:
temp_df = (payment_df.drop("payment_month"))
                    # .withColumn("amount", F.round(F.col("amount")*0.5, 2)))

## Register Delta table to Hive Metastore

In [34]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS streamify.delta_payment (
        rental_id integer,
        payment_date string,
        payment_id integer,
        customer_id integer,
        amount double,
        staff_id integer
    )
    USING DELTA
    PARTITIONED BY(payment_year integer)
    LOCATION 's3a://lakehouse/streamify.db/delta_payment'
""")

spark.sql("SELECT * FROM streamify.delta_payment").count()
# spark.sql("DESCRIBE EXTENDED streamify.delta_payment").show()

128

In [33]:
(temp_df.write
        .format("delta")
        .partitionBy("payment_year")
        .option("path", "s3a://lakehouse/streamify.db/delta_payment")
        .mode("append")
        .save())

**How to write streaming data to Delta table**

```python
# Cấu hình streaming query
write_stream = (payment_df.writeStream
                        .format("delta")
                        .partitionBy(partitioned_col)
                        .option("checkpointLocation", checkpoint_path)
                        .option("path", storage_path)
                        .trigger(processingTime=trigger)
                        .outputMode(output_mode)
                        .start())
```

## Register Hive table to Hive Metastore

In [ ]:
(temp_df.write
        .mode("append")
        .insertInto("streamify.hive_payment"))

In [ ]:
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS streamify.delta_payment (
        rental_id integer,
        payment_date string,
        payment_id integer,
        customer_id integer,
        amount double,
        staff_id integer
    )
    PARTITIONED BY(payment_year integer)
    LOCATION 's3a://lakehouse/streamify.db/hive_payment'
""")

spark.sql("SELECT * FROM streamify.hive_payment").show(5)
spark.sql("DESCRIBE EXTENDED streamify.hive_payment").show()

**How to write streaming data to Hive table**

```python
def write_to_hive_table(batch_df, batch_id):
    # Ghi dữ liệu vào Hive table (append mode)
    (batch_df.write
            .mode("append")
            .insertInto("streamify.hive_payment"))

# Cấu hình streaming query
write_stream = (payment_df.writeStream
                        .foreachBatch(write_to_hive_table)
                        .option("checkpointLocation", checkpoint_path)
                        .trigger(processingTime=trigger)
                        .outputMode(output_mode)
                        .start())
```

In [ ]:
spark.sql("SHOW SCHEMAS FROM hive")

In [5]:
spark.stop()